# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [152]:
import warnings
warnings.filterwarnings('ignore')

In [153]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [154]:
#!pip install pandas

In [155]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [156]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [157]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [158]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.7)


In [159]:
prompt = ChatPromptTemplate.from_template("Describe the features and benefits of the product: {product}")
 

In [160]:

chain = LLMChain(llm=llm, prompt=prompt)

In [161]:
product = "Queen Size Sheet Set"
response = chain.run(product)
print("Description for product:", response)

Description for product: The Queen Size Sheet Set is a bedding essential that includes a flat sheet, fitted sheet, and pillowcases specifically designed to fit a queen-sized bed. 

Features:
1. High-quality material: The sheet set is made of soft and luxurious fabric, such as cotton, microfiber, or bamboo, ensuring a comfortable and cozy night's sleep.
2. Variety of colors and designs: The sheet set comes in a range of colors and patterns to suit any bedroom decor style.
3. Easy care: The sheets are machine washable and easy to care for, making them convenient for everyday use.
4. Deep pockets: The fitted sheet has deep pockets to ensure a secure and snug fit on the mattress, preventing it from slipping off during the night.
5. Durable construction: The sheets are designed to withstand regular use and washing, maintaining their quality and softness over time.

Benefits:
1. Enhanced comfort: The soft and breathable fabric of the sheet set provides a comfortable sleeping experience, help

## SimpleSequentialChain

In [162]:
from langchain.chains import SimpleSequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [163]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template("Describe the key features of the product: {product}")


# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [164]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Using the features: {input}, write a marketing tagline for this product."
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [165]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)


In [166]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
- Queen size sheet set typically includes one flat sheet, one fitted sheet, and two standard pillowcases
- Made from soft and durable materials such as cotton, microfiber, or linen
- Available in a variety of colors and patterns to suit any bedroom decor
- Fitted sheet has deep pockets to accommodate mattresses up to 18 inches thick
- Sheets are machine washable for easy care and maintenance
- Provides a comfortable and luxurious sleeping surface
- Queen size sheet sets are designed to fit a standard queen size bed, measuring approximately 60 inches by 80 inches
- Can be purchased individually or as part of a bedding bundle to complete the look of your bedroom
"Indulge in luxury and style with our queen size sheet sets - the perfect blend of comfort, durability, and design for a dreamy night's sleep. Elevate your bedroom decor with our wide range of colors and patterns. Fits like a dream on mattresses up to 18 inches thick. Easy care and 

'"Indulge in luxury and style with our queen size sheet sets - the perfect blend of comfort, durability, and design for a dreamy night\'s sleep. Elevate your bedroom decor with our wide range of colors and patterns. Fits like a dream on mattresses up to 18 inches thick. Easy care and machine washable for your convenience. Complete your bedroom oasis with our premium quality sheet sets. Sweet dreams await!"'

In [167]:
# Run the chain for a product
product = "Luxury Air Mattress"
result = overall_simple_chain.run(product)
print("Marketing tagline for product:", result)

# Repeat for another product
product = "Milk Frother Handheld"
result = overall_simple_chain.run(product)
print("Marketing tagline for product:", result)



> Entering new SimpleSequentialChain chain...
1. High-quality materials: A luxury air mattress is typically made with top-of-the-line materials such as premium PVC or TPU for durability and comfort.

2. Comfort and support: Luxury air mattresses are designed to provide optimal comfort and support, with features such as multiple air chambers for customized firmness levels and built-in pillow tops for added cushioning.

3. Adjustable firmness: Many luxury air mattresses come with adjustable firmness settings, allowing users to customize the level of support to their liking.

4. Built-in pump: Most luxury air mattresses come with a built-in pump for easy and quick inflation and deflation, eliminating the need for an external pump.

5. Durability: Luxury air mattresses are designed to be long-lasting and resistant to punctures and leaks, ensuring years of reliable use.

6. Portability: Despite their size and quality, luxury air mattresses are typically lightweight and easy to transport, 

**Repeat the above twice for different products**

## SequentialChain

In [168]:
from langchain.chains import SequentialChain
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [169]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template("Translate this review to French: {review}")
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="translated_review")


In [170]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in one sentence: {translated_review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

In [171]:
third_prompt = ChatPromptTemplate.from_template(
    "Translate the following summary back to English: {summary}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="english_summary")


In [172]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up message to the customer based on this summary: {english_summary}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="follow_up_message")


In [173]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],  # Input to the overall chain
    output_variables=["translated_review", "summary", "english_summary", "follow_up_message"],
    verbose=True
)


In [174]:
review = df.Review[5]
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste poor. The foam doesn't last, it's strange. I buy the same ones in stores and the taste is much better...\nOld batch or counterfeit!?",
 'summary': 'The reviewer is disappointed with the poor taste and lack of lasting foam in the product, suspecting it may be an old batch or counterfeit.',
 'english_summary': "The reviewer is disappointed with the product's bad taste and the fact that it does not have long-lasting foam. They suspect that it may be an old batch or counterfeit.",
 'follow_up_message': 'Dear Valued Customer,\n\nWe are sorry to hear about your disappointing experience with our product. Your feedback is very important to us, and we take your concerns seriously. We can assure you that we only use high-quality ingredients in our products and do not sell counter

**Repeat the above twice for different products or reviews**

## Router Chain

In [186]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [187]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [188]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [189]:
llm = ChatOpenAI(temperature=0)

In [191]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [192]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [193]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [194]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [195]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [196]:
result = chain.run("What is black body radiation?")
print("Physics response:", result)



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.
Physics response: Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all radiation that falls on it and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and depends only on the temperature of the black body. This phenomenon is described by Planck's law, which states that the intensity of radiation emitted by a black body at a given wavelength is proportional to the temperature of the body and the wavelength raised to the fifth power.


In [197]:
result = chain.run("What is 2 + 2?")
print("Math response:", result)



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2 + 2?'}
> Finished chain.
Math response: The answer to 2 + 2 is 4.


In [ ]:
result = chain.run("Explain how DNA works in the human body.")
print("Biology response:", result)

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [199]:
result = chain.run("What is the time complexity of quicksort?")
print("Computer Science response:", result)



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the time complexity of quicksort?'}
> Finished chain.
Computer Science response: The time complexity of quicksort is O(n log n) on average, where n is the number of elements in the array being sorted. This is because quicksort divides the array into smaller subarrays and recursively sorts them, with each division taking O(n) time and there being O(log n) divisions. However, in the worst-case scenario, quicksort can have a time complexity of O(n^2) if the pivot element chosen is consistently the smallest or largest element in the array.


In [200]:
result = chain.run("Explain the causes of World War II.")
print("History response:", result)



> Entering new MultiPromptChain chain...
History: {'input': 'Explain the causes of World War II.'}
> Finished chain.
History response: World War II was caused by a combination of factors that had been building up over several decades. Some of the main causes include:

1. Treaty of Versailles: The harsh terms imposed on Germany after World War I, including massive reparations payments and territorial losses, created a sense of resentment and humiliation among the German people. This led to the rise of Adolf Hitler and the Nazi Party, who promised to restore Germany's power and prestige.

2. Expansionist policies: In the 1930s, both Germany and Japan pursued aggressive expansionist policies, seeking to conquer territory and resources in order to fuel their economies and establish themselves as dominant powers in their respective regions.

3. Failure of appeasement: The policy of appeasement pursued by Western powers, particularly Britain and France, in response to German aggression onl